# Get Borrows

In this notebook, we compile the data for borrow transactions from the API.

In [1]:
#import packages
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import matplotlib.dates as md
import math
import time

In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_tes

## Fetch Borrow Transaction Data

First, if this data was previously compiled, we load the existing data.

In [2]:
start_time = 0
try:
    old_data = pd.read_csv('borrows.csv')
    exists = True
    start_time = old_data['timestamp'].max()
    old_data.info()
except:
    old_data = pd.DataFrame()
    exists = False

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127617 entries, 0 to 127616
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   amount           127617 non-null  float64
 1   borrowRate       127617 non-null  float64
 2   borrowRateMode   127617 non-null  object 
 3   onBehalfOf       127617 non-null  object 
 4   pool             127617 non-null  object 
 5   reserve          127617 non-null  object 
 6   timestamp        127617 non-null  int64  
 7   user             127617 non-null  object 
 8   type             127617 non-null  object 
 9   reservePriceETH  127617 non-null  float64
 10  reservePriceUSD  127617 non-null  float64
 11  amountETH        127617 non-null  float64
 12  amountUSD        127617 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 12.7+ MB


Here, we write a query to fetch transaction information from the API.

In [3]:
lastId='""'
borrow_data=[]
#loop until no more data left
while(1):
    try:
        #set query
        query="""
        {
            borrows (first: 1000 orderBy: id where:{id_gt:"""+lastId+""", timestamp_gt:"""+str(start_time)+"""}) {
            id,
            user{
                id
            }
            onBehalfOf{
                id
            }
            pool{
                id
            }
            amount,
            reserve {
              id,
              symbol
            },
            borrowRate,
            borrowRateMode,
            timestamp
            }
        }
        """
        #make request
        url = 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
        request = requests.post(url,json={'query':query})
        #store data
        borrow_data.extend(request.json()['data']['borrows'])
        lastId = "\""+request.json()['data']['borrows'][-1]['id']+"\""
    except Exception as e:
        #exit when no more data left to get
        print(e)
        break

#create borrows data frame
df_borrows = pd.DataFrame(borrow_data)
df_borrows['type']='borrow'
df_borrows.info()

list index out of range
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3649 entries, 0 to 3648
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              3649 non-null   object
 1   user            3649 non-null   object
 2   onBehalfOf      3649 non-null   object
 3   pool            3649 non-null   object
 4   amount          3649 non-null   object
 5   reserve         3649 non-null   object
 6   borrowRate      3649 non-null   object
 7   borrowRateMode  3649 non-null   object
 8   timestamp       3649 non-null   int64 
 9   type            3649 non-null   object
dtypes: int64(1), object(9)
memory usage: 285.2+ KB


### Re-Format Borrow Transaction Data

Some of the values in the dataset are dictionaries, so we must re-format the data to get the wanted field from those dictionaries.

In [4]:
#get id's
def getUser(row):
    return row['user']['id']
df_borrows['user']=df_borrows.apply(lambda x: getUser(x), axis=1)

def getOnBehalfOf(row):
    if not isinstance(row['onBehalfOf'],float):
        return row['onBehalfOf']['id']
    else:
        return np.nan
df_borrows['onBehalfOf']=df_borrows.apply(lambda x: getOnBehalfOf(x), axis=1)

def getPool(row):
    return row['pool']['id']
df_borrows['pool']=df_borrows.apply(lambda x: getPool(x), axis=1)

#get symbols
def getReserve(row):
    if not isinstance(row['reserve'],float):
        return row['reserve']['symbol']
    else:
        return np.nan
df_borrows['reserve']=df_borrows.apply(lambda x: getReserve(x), axis=1)

### Fetch Price Data

The API does not list the price of the asset for each transaction, so we must gather this information in another way. We can search for the most recent price of the desired asset before the transaction occurred. Prices are reported in Ether, not USD. Because of this, we also get the price of Tether at the time of the transaction. Tether is a stable coin, so its price should always be close to one dollar. We will divide the asset price by the price of Tether to determine the price of the asset in USD. 

In [5]:
pricesSym=[]
pricesUSDT=[]
i=0
#get prices for each asset at time of transaction, and price for USDT at time of transaction
def getPrice(row, sym):
    global i
    
    #get symbol and time
    symbol = row[sym]
    timestamp = row['timestamp']
    
    #there doesn't seem to be price data for ammWETH, so instead we will just substitute WETH as they have the same price
    if symbol =='AmmWETH':
        symbol='WETH'
    
    #get query
    query="""
    {
    reserves(where: { symbol_in:["USDT",\""""+symbol+"""\"] }){
        symbol,
        price{
            priceInEth,
            priceHistory(where:{timestamp_lte: """+str(timestamp)+"""} orderBy: timestamp orderDirection: desc first:1){
                price,
                timestamp
              }
            }
          }
        }    
    """
    #keep trying request until it is successful
    while(True):
        try:
            #get json
            url = 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
            request = requests.post(url,json={'query':query})
            req_json = request.json()
            break
        except:
            #if request unsuccessful, try again in 10 seconds
            print('stalling')
            time.sleep(10)
 
    try:
        #if only data for 1 asset...
        if len(req_json['data']['reserves'])<2:
            #if USDT, add data
            if symbol=="USDT":
                pricesSym.append(req_json['data']['reserves'][0]['price']['priceHistory'][0]['price'])
                pricesUSDT.append(req_json['data']['reserves'][0]['price']['priceHistory'][0]['price'])
            #otherwise, symbol not found
            else:
                pricesSym.append(np.nan)
                pricesUSDT.append(np.nan)
        #if both present...
        else:
            #ensure price data exists for asset
            phistory = req_json['data']['reserves'][0]['price']['priceHistory']
            #if data not available, set as null
            if len(phistory)==0:
                #WETH doesn't seem to return a priceHistory list, so we the priceInETH column 
                #(as the price in ETH for WETH is always 1)
                if symbol=='WETH' or symbol=='AmmWETH':
                    pricesSym.append(req_json['data']['reserves'][0]['price']['priceInEth'])
                    pricesUSDT.append(req_json['data']['reserves'][1]['price']['priceHistory'][0]['price'])
                else:
                    pricesSym.append(np.nan)
                    pricesUSDT.append(req_json['data']['reserves'][1]['price']['priceHistory'][0]['price'])
            #otherwise add data
            else:
                pricesSym.append(phistory[0]['price'])
                pricesUSDT.append(req_json['data']['reserves'][1]['price']['priceHistory'][0]['price'])
    except:
        print('ERROR')
        print(req_json)
        return
    
    #update progress
    i+=1
    if i%5000==0:
        print(i)

#get borrow prices
print('getting borrow prices...')
df_borrows.apply(lambda x: getPrice(x,'reserve'),axis=1)

df_borrows['priceSym']=pricesSym
df_borrows['priceUSDT']=pricesUSDT

getting borrow prices...


### Get Decimal Data

The amount value is reported in units of the lowest denomination for each currency. To standardize these values, we must know the number of decimals each currency is reported in. The following query creates a dictionary holding the number of decimals for each currency.

In [6]:
#set query
query="""
        {
  reserves(first:1000){
    symbol
    decimals
  }
}
        """
#make request
url = 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
request = requests.post(url,json={'query':query})
jsondata=request.json()['data']['reserves']

#create dictionary of the number of decimals in each asset
decimals=dict()
for data in jsondata:
    decimals[data['symbol']]=int(data['decimals'])
    
decimals

{'TUSD': 18,
 'AmmUniWBTCUSDC': 18,
 'RAI': 18,
 'GUSD': 2,
 'YFI': 18,
 'BAT': 18,
 'MANA': 18,
 'DPI': 18,
 'AmmBptWBTCWETH': 18,
 'UNI': 18,
 'AmmWBTC': 8,
 'WBTC': 8,
 'AmmUniYFIWETH': 18,
 'AmmUniCRVWETH': 18,
 'REN': 18,
 'AmmUniSNXWETH': 18,
 'BUSD': 18,
 'AmmGUniDAIUSDC': 18,
 'LINK': 18,
 'SUSD': 18,
 'AmmBptBALWETH': 18,
 'AmmDAI': 18,
 'DAI': 18,
 'AAVE': 18,
 'FRAX': 18,
 'XSUSHI': 18,
 'AmmUniRENWETH': 18,
 'PAX': 18,
 'FEI': 18,
 'MKR': 18,
 'AmmUSDC': 6,
 'USDC': 6,
 'AmmUniLINKWETH': 18,
 'AmmUniDAIWETH': 18,
 'AmmUniDAIUSDC': 18,
 'STETH': 18,
 'AmmUniUSDCWETH': 18,
 'AmmUniBATWETH': 18,
 'BAL': 18,
 'AmmUniWBTCWETH': 18,
 'SNX': 18,
 'AmmWETH': 18,
 'WETH': 18,
 'AmmUniMKRWETH': 18,
 'AmmGUniUSDCUSDT': 18,
 'AmmUniUNIWETH': 18,
 'AMPL': 9,
 'RENFIL': 18,
 'CRV': 18,
 'AmmUSDT': 6,
 'USDT': 6,
 'KNC': 18,
 'AmmUniAAVEWETH': 18,
 'ZRX': 18,
 'ENJ': 18}

### Adjust Amount

Next, we convert the amount column to type float. Then, with the information we gathered above, we write a function to adjust the transaction amounts. 

In [7]:
#transform amount column to float
df_borrows['amount']=df_borrows['amount'].astype(float)

In [8]:
#function to divide each amount based on the reserve
def adjustAmount(row):
    decs = decimals[row['reserve']]
    return row['amount']/(10**decs)

#adjust amounts
df_borrows['amount']=df_borrows.apply(lambda x: adjustAmount(x),axis=1)
df_borrows['amount'].describe()

count    3.649000e+03
mean     4.554424e+05
std      4.397129e+06
min      1.010294e-11
25%      1.800000e+03
50%      1.200000e+04
75%      7.500000e+04
max      1.250000e+08
Name: amount, dtype: float64

### Re-Format Price Data

We format the raw price data so that the prices are reported in USD. We will also determine the total amount of the transaction in USD.

In [9]:
#get the prices in ethereum and in USD
df_borrows['reservePriceETH'] = df_borrows['priceSym'].astype(float)
df_borrows['reservePriceUSD'] = df_borrows['reservePriceETH']*(1/(df_borrows['priceUSDT'].astype(float)))
df_borrows['reservePriceETH'] = df_borrows['reservePriceETH']/1e18

#get amount in ETH
df_borrows['amountETH'] = df_borrows['amount'].astype(float)*df_borrows['reservePriceETH']

#get amount in USD
df_borrows['amountUSD']=df_borrows['amount'].astype(float)*df_borrows['reservePriceUSD']                                     
                 
#reduce borrow rate to percent
df_borrows['borrowRate']=df_borrows['borrowRate'].astype(float)/1e27*100                 
                                               
#drop redundent columns
df_borrows.drop(columns=['priceSym','priceUSDT'],inplace=True)
                                               
#reset index
df_borrows = df_borrows.set_index('id')                                     

### Save Final Data Frame

Finally, we save the dataframe to a csv file.

In [10]:
df_borrows = old_data.append(df_borrows)
df_borrows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 131266 entries, 0 to 0xffae32d41f70f9738e60b1f24198057afcc6fb1fef0042512e5df831c6825e1c:2
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   amount           131266 non-null  float64
 1   borrowRate       131266 non-null  float64
 2   borrowRateMode   131266 non-null  object 
 3   onBehalfOf       131266 non-null  object 
 4   pool             131266 non-null  object 
 5   reserve          131266 non-null  object 
 6   timestamp        131266 non-null  int64  
 7   user             131266 non-null  object 
 8   type             131266 non-null  object 
 9   reservePriceETH  131266 non-null  float64
 10  reservePriceUSD  131266 non-null  float64
 11  amountETH        131266 non-null  float64
 12  amountUSD        131266 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 14.0+ MB


In [11]:
df_borrows.to_csv("borrows.csv",index=False)

In [12]:
pd.read_csv('borrows.csv')

,amount,borrowRate,borrowRateMode,onBehalfOf,pool,reserve,timestamp,user,type,reservePriceETH,reservePriceUSD,amountETH,amountUSD
0,5.000000e+03,10.937930,Stable,0xf1fa3b44c153f7988ff5de730b138eec476a7533,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1639138209,0xf1fa3b44c153f7988ff5de730b138eec476a7533,borrow,0.000240,1.002934,1.200419,5.014671e+03
1,1.500000e+01,0.259066,Variable,0xc3b1c1c3dc334a283c64fe3fa2e9c69a0d022357,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,WETH,1633275840,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,borrow,1.000000,3421.870819,15.000000,5.132806e+04
2,4.150163e+04,6.274937,Variable,0x94ee9c600870c4199a1af8496eeb3087f2d1c32f,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,DAI,1621340435,0x94ee9c600870c4199a1af8496eeb3087f2d1c32f,borrow,0.000285,0.994804,11.840000,4.128600e+04
3,7.000000e+06,2.589628,Variable,0x51346d389ec7adf11c5b96695ef003dbb849af9a,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1622477822,0x51346d389ec7adf11c5b96695ef003dbb849af9a,borrow,0.000381,1.000000,2668.984254,7.000000e+06
4,1.500000e+04,8.802541,Variable,0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1619775984,0x416d7f3823bc7d8dbab00056ede0325e52f0a5c2,borrow,0.000361,1.004339,5.416500,1.506508e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131261,4.200000e+04,3.146646,Variable,0x2fe09e93acbb8b0da86c394335b8a92d3f5e273e,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1644506169,0x2fe09e93acbb8b0da86c394335b8a92d3f5e273e,borrow,0.000313,1.000000,13.163456,4.200000e+04
131262,5.265141e+04,2.659370,Variable,0xed1cb75b67ea20cfe575645c52b953437b194793,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDC,1646143950,0xed1cb75b67ea20cfe575645c52b953437b194793,borrow,0.000333,0.999888,17.511981,5.264553e+04
131263,1.000000e+01,0.371007,Variable,0x23855dfd30b46ca5590221cc0612e48fa3e1d829,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,WETH,1644519470,0xcc9a0b7c43dc2a5f023bb9b738e45b0ef6b06e04,borrow,1.000000,3190.594568,10.000000,3.190595e+04
131264,1.000000e+04,3.157489,Variable,0xe9177c5204f633a562747010788903747a858dcd,0xb53c1a33016b2dc2ff3653530bff1848a515c8c5,USDT,1644486628,0xe9177c5204f633a562747010788903747a858dcd,borrow,0.000314,1.000000,3.135814,1.000000e+04
